In [1]:
#install
!pip install ortools
!pip install openrouteservice
!pip install folium
!pip install geopy

#library
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
import folium
import openrouteservice
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
from tqdm import tqdm
from openrouteservice import distance_matrix
from ortools.constraint_solver import pywrapcp, routing_enums_pb2
from ortools.sat.python import cp_model

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: absl-py
    Found existing installation: absl-py 1.4.0
    Uninstalling absl-py-1.4.0:
      Successfully uninstalled absl-py-1.4.0


In [2]:
# !wget https://github.com/linneverh/logistics/releases/download/v1/SOTE.-.Orders.xlsx

In [3]:
# Load the Excel file
#df = pd.read_excel('SOTE.-.Orders.xlsx')

In [4]:
#df.head()

## New Distance Matrix

In [5]:
#Load updated matrix

# URLs of the two CSV files on GitHub
url1 = 'https://raw.githubusercontent.com/linneverh/logistics/main/numerical_clusters_driving_distance_matrix_km.csv'
url2 = 'https://raw.githubusercontent.com/linneverh/logistics/main/numerical_cluster_info.csv'

In [6]:
# Dataframe with corresponding capacities
capacities_31012025 = pd.read_csv(url2)
print(capacities_31012025)

    Cluster  Capacity Value Weight
0         0                    NaN
1         1               4854.893
2         2                313.848
3         3               3061.860
4         4               2919.247
5         5                 53.686
6         6               1122.641
7         7                418.818
8         8                478.516
9         9                236.506
10       10                216.452
11       11                350.088
12       12                101.158
13       13                 54.631
14       14                    NaN
15       15                163.592
16       16                126.910
17       17                  4.950
18       18                 61.096
19       19                129.648
20       20                172.214
21       21                 24.025
22       22                 15.948
23       23                 45.609
24       24                  2.479
25       25                134.100
26       26                160.594
27       27         

In [7]:
# Driving distance maxtrix
driving_distance = pd.read_csv(url1)

# Names for row and columns
# Set 'Unnamed: 0' as the index
driving_distance = driving_distance.set_index('Unnamed: 0')
driving_distance.index.name = None  # Remove the index name

# Explicitly set the column names to match the index names
driving_distance.columns = driving_distance.index

# Now, the index and column names are guaranteed to be the same cluster numbers
# Let's verify with a print
print("Driving distance matrix with matching index and column names:")
display(driving_distance)

# Keep the rest of the code that filters based on demand
# Get the correct cluster order from the distance matrix (this is now the index)
ordered_clusters = driving_distance.index.tolist()

# Filter demands based on the original ordered clusters
# Reorder the capacity info to match
capacities_ordered = capacities_31012025.set_index('Cluster').loc[ordered_clusters].reset_index()

# Fill missing values with 0 and fix depot demand
capacities_ordered['Capacity Value Weight'] = capacities_ordered['Capacity Value Weight'].fillna(0)
capacities_ordered.loc[capacities_ordered['Cluster'] == 0, 'Capacity Value Weight'] = 0

# Create the demand vector in the correct order
demand = capacities_ordered['Capacity Value Weight'].round().astype(int).tolist()

# Only keep clusters with demand > 0 or depot (cluster 0)
clusters_to_keep = [cluster for cluster, d in zip(ordered_clusters, demand) if d > 0 or cluster == 0]

# Convert clusters_to_keep to a pandas Index for easier alignment
clusters_index = pd.Index(clusters_to_keep)

# Rebuild driving_distance and capacity vectors using reindex
# This should now work correctly as index and columns of driving_distance match
filtered_distance = driving_distance.reindex(index=clusters_index, columns=clusters_index, fill_value=0)

# Filter the demand vector to match the order of clusters in filtered_distance
demand_map = dict(zip(ordered_clusters, demand))
filtered_demand = [demand_map[cluster] for cluster in filtered_distance.index] # Use the index of the filtered_distance DataFrame

print("\nFiltered distance matrix:")
display(filtered_distance)
print("\nFiltered demand vector:")
print(filtered_demand)

# The rest of your code (create_data_model, print_solution, main) can remain as in the previous suggestion
# as it correctly uses filtered_distance and filtered_demand

Driving distance matrix with matching index and column names:


,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,50,51,52,53
0,0.00,911.23,128.64,545.59,905.42,228.08,1599.17,1307.00,466.71,370.42,...,889.90,1013.00,995.56,918.63,889.35,450.18,1424.28,725.89,2830.84,253.41
1,921.70,0.00,809.00,1455.41,1563.76,1138.40,1099.77,807.59,1377.04,1280.75,...,247.34,1923.33,1905.89,130.52,85.30,1061.67,2334.09,1635.71,3740.66,714.55
2,130.19,797.22,0.00,663.89,1024.24,346.89,1485.16,1192.99,585.53,489.24,...,775.89,1131.82,1114.38,804.62,775.34,445.65,1542.58,844.19,2949.15,139.40
3,545.66,1445.45,662.86,0.00,795.61,459.46,2133.39,1841.22,426.10,234.24,...,1424.12,744.03,726.60,1452.85,1423.57,781.79,882.54,184.15,2289.11,787.63
4,907.08,1571.23,1024.78,793.45,0.00,768.87,2259.17,1966.99,460.58,682.45,...,1537.10,651.11,334.87,1578.62,1549.35,507.33,1233.58,973.75,2466.08,1055.12
5,226.12,1126.41,343.82,460.71,767.18,0.00,1814.35,1522.17,328.48,244.58,...,1105.08,887.15,869.72,1133.80,1104.53,407.71,1339.40,641.01,2745.96,468.58
6,1610.39,1101.29,1497.68,2144.09,2252.44,1827.09,0.00,299.34,2065.73,1969.44,...,1206.81,2612.01,2594.58,1180.55,1039.65,1750.36,3022.78,2324.39,4429.35,1403.23
7,1317.00,807.91,1204.30,1850.71,1959.06,1533.71,299.71,0.00,1772.35,1676.06,...,913.43,2318.63,2301.20,887.17,746.27,1456.98,2729.40,2031.01,4135.97,1109.85
8,482.22,1382.50,599.92,404.31,459.90,344.01,2070.45,1778.27,0.00,257.59,...,1361.18,664.15,644.11,1389.90,1360.63,367.25,1283.00,584.61,2707.72,724.68
9,365.24,1265.53,482.94,235.25,682.71,238.35,1953.47,1661.29,244.00,0.00,...,1244.20,661.70,644.26,1272.92,1243.65,599.69,1113.94,415.55,2520.50,607.70



Filtered distance matrix:


,0,1,2,3,4,5,6,7,8,9,...,29,30,31,35,36,37,41,42,46,52
0,0.00,911.23,128.64,545.59,905.42,228.08,1599.17,1307.00,466.71,370.42,...,1023.98,829.17,1850.45,369.88,2599.94,274.87,889.90,1013.00,450.18,2830.84
1,921.70,0.00,809.00,1455.41,1563.76,1138.40,1099.77,807.59,1377.04,1280.75,...,1933.80,82.66,2760.27,1188.15,3509.76,638.66,247.34,1923.33,1061.67,3740.66
2,130.19,797.22,0.00,663.89,1024.24,346.89,1485.16,1192.99,585.53,489.24,...,1142.29,715.16,1968.76,384.60,2718.24,160.86,775.89,1131.82,445.65,2949.15
3,545.66,1445.45,662.86,0.00,795.61,459.46,2133.39,1841.22,426.10,234.24,...,482.25,1363.39,1308.72,756.81,2058.20,809.09,1424.12,744.03,781.79,2289.11
4,907.08,1571.23,1024.78,793.45,0.00,768.87,2259.17,1966.99,460.58,682.45,...,833.28,1489.16,1659.75,1275.76,2239.56,1065.20,1537.10,651.11,507.33,2466.08
5,226.12,1126.41,343.82,460.71,767.18,0.00,1814.35,1522.17,328.48,244.58,...,939.10,1044.34,1765.57,594.80,2515.06,490.04,1105.08,887.15,407.71,2745.96
6,1610.39,1101.29,1497.68,2144.09,2252.44,1827.09,0.00,299.34,2065.73,1969.44,...,2622.49,1091.09,3448.96,1876.84,4198.44,1327.35,1206.81,2612.01,1750.36,4429.35
7,1317.00,807.91,1204.30,1850.71,1959.06,1533.71,299.71,0.00,1772.35,1676.06,...,2329.11,797.71,3155.58,1583.45,3905.06,1033.97,913.43,2318.63,1456.98,4135.97
8,482.22,1382.50,599.92,404.31,459.90,344.01,2070.45,1778.27,0.00,257.59,...,882.70,1300.44,1709.17,850.90,2481.19,746.14,1361.18,664.15,367.25,2707.72
9,365.24,1265.53,482.94,235.25,682.71,238.35,1953.47,1661.29,244.00,0.00,...,713.65,1183.46,1540.11,733.92,2289.60,629.16,1244.20,661.70,599.69,2520.50



Filtered demand vector:
[0, 4855, 314, 3062, 2919, 54, 1123, 419, 479, 237, 216, 350, 101, 55, 164, 127, 5, 61, 130, 172, 24, 16, 46, 2, 134, 161, 2, 23, 60, 81, 159, 41, 162, 5, 23, 14, 183, 91]


In [22]:
print("Clusters in use:", clusters_to_keep)
print("Total demand:", sum(filtered_demand))

Clusters in use: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 35, 36, 37, 41, 42, 46, 52]
Total demand: 16070


## First Capacitated Facility Location Problem

In [8]:
def create_data_model():
    data = {}
    data['distance_matrix'] = filtered_distance.values.tolist()
    data['demands'] = filtered_demand
    data['vehicle_capacities'] = [16000] * 4  # Adjust as needed
    data['num_vehicles'] = 4
    data['depot'] = 0
    return data

In [23]:
def print_solution(data, manager, routing, solution):
    """Prints routes and returns total distance."""
    total_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        route_distance = 0
        capacity_used = 0
        plan_output = f'\n🚚 Route for vehicle {vehicle_id}:\n'

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            demand = data['demands'][node_index]
            capacity_used += demand
            plan_output += f' → Node {node_index} (Demand: {demand}, Used: {capacity_used})\n'
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

        plan_output += f' → Return to depot\n'
        plan_output += f'Distance: {route_distance} km\n'
        print(plan_output)
        total_distance += route_distance

    print(f'\n📦 Total distance: {total_distance} km')
    return total_distance

In [10]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

def main():
    data = create_data_model()

    # Create the routing index manager
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'],
                                           data['depot'])

    # Create Routing Model
    routing = pywrapcp.RoutingModel(manager)

    # Define distance callback
    def distance_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return int(data['distance_matrix'][from_node][to_node])

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Define demand callback
    def demand_callback(from_index):
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)

    # Add capacity constraint
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # no slack
        data['vehicle_capacities'],
        True,  # start cumul to zero
        'Capacity'
    )

    # Add distance constraint
    routing.AddDimension(
        transit_callback_index,
        0,
        6000,  # max distance per vehicle
        True,
        'Distance'
    )
    distance_dimension = routing.GetDimensionOrDie('Distance')
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Solver parameters
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve
    solution = routing.SolveWithParameters(search_parameters)

    if solution:
        return print_solution(data, manager, routing, solution)
    else:
        print("No solution found.")
        return None


In [24]:
baseline_total_distance = main()


🚚 Route for vehicle 0:
 → Node 0 (Demand: 0, Used: 0)
 → Node 32 (Demand: 162, Used: 162)
 → Node 22 (Demand: 46, Used: 208)
 → Node 37 (Demand: 91, Used: 299)
 → Node 25 (Demand: 161, Used: 460)
 → Return to depot
Distance: 5711 km


🚚 Route for vehicle 1:
 → Node 0 (Demand: 0, Used: 0)
 → Node 2 (Demand: 314, Used: 314)
 → Node 33 (Demand: 5, Used: 319)
 → Node 11 (Demand: 350, Used: 669)
 → Node 23 (Demand: 2, Used: 671)
 → Node 26 (Demand: 2, Used: 673)
 → Node 6 (Demand: 1123, Used: 1796)
 → Node 19 (Demand: 172, Used: 1968)
 → Node 7 (Demand: 419, Used: 2387)
 → Node 1 (Demand: 4855, Used: 7242)
 → Node 29 (Demand: 81, Used: 7323)
 → Node 34 (Demand: 23, Used: 7346)
 → Node 17 (Demand: 61, Used: 7407)
 → Node 31 (Demand: 41, Used: 7448)
 → Return to depot
Distance: 4647 km


🚚 Route for vehicle 2:
 → Node 0 (Demand: 0, Used: 0)
 → Node 14 (Demand: 164, Used: 164)
 → Node 3 (Demand: 3062, Used: 3226)
 → Node 24 (Demand: 134, Used: 3360)
 → Node 10 (Demand: 216, Used: 3576)
 → Nod

## FLP

In [14]:
def run_flp(distance_matrix, demand_vector, cluster_ids, depot_index=0, max_warehouses=2):
    num_nodes = len(distance_matrix)
    EF = 1  # scaling factor; keep 1 if just minimizing distance

    model = cp_model.CpModel()

    # Binary: is warehouse opened at node i?
    open_vars = [model.NewBoolVar(f"open[{i}]") for i in range(num_nodes)]

    # Binary: is node j served by warehouse at i?
    assign = [[model.NewBoolVar(f"assign[{i}][{j}]") for j in range(num_nodes)] for i in range(num_nodes)]

    # Each node must be assigned to one warehouse
    for j in range(num_nodes):
        model.Add(sum(assign[i][j] for i in range(num_nodes)) == 1)

    # Can only assign to open warehouses
    for i in range(num_nodes):
        for j in range(num_nodes):
            model.Add(assign[i][j] <= open_vars[i])

    # Fix depot (cluster 0) to be open
    model.Add(open_vars[depot_index] == 1)

    # Open exactly max_warehouses (1 depot + 1 new)
    model.Add(sum(open_vars) == max_warehouses)

    # Objective: minimize total emissions (distance × demand)
    total_cost_terms = []
    for i in range(num_nodes):
        for j in range(num_nodes):
            cost = int(distance_matrix[i][j] * demand_vector[j] * EF)
            total_cost_terms.append(assign[i][j] * cost)

    model.Minimize(sum(total_cost_terms))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status in (cp_model.OPTIMAL, cp_model.FEASIBLE):
        selected_warehouses = [i for i in range(num_nodes) if solver.Value(open_vars[i])]
        assignments = [max(range(num_nodes), key=lambda i: solver.Value(assign[i][j])) for j in range(num_nodes)]

        print("✅ Warehouses selected at clusters:", [cluster_ids[i] for i in selected_warehouses])
        return selected_warehouses, assignments
    else:
        print("❌ No solution found.")
        return None, None


In [15]:
selected_warehouses, assignments = run_flp(
    filtered_distance.values.tolist(),
    filtered_demand,
    cluster_ids=clusters_to_keep,
    depot_index=0,          # Cluster 0 is fixed as open
    max_warehouses=2        # Total 2 warehouses (1 existing + 1 new)
)

✅ Warehouses selected at clusters: [0, 1]


In [16]:
from collections import defaultdict

def group_assignments_by_warehouse(assignments, selected_warehouses):
    clusters_by_wh = defaultdict(list)
    for node_idx, wh_idx in enumerate(assignments):
        clusters_by_wh[wh_idx].append(node_idx)
    return clusters_by_wh

# Returns: dict { warehouse_index -> list of node indices assigned }
assignments_by_wh = group_assignments_by_warehouse(assignments, selected_warehouses)

## Re-run CVRP

In [17]:
def run_cvrp_for_warehouse(wh_index, assigned_node_indices, full_distance_df, full_demand_vector, cluster_ids):
    # Step 1: remap so warehouse is first
    nodes = [wh_index] + [i for i in assigned_node_indices if i != wh_index]

    # Step 2: subset distance and demand
    sub_matrix = full_distance_df.iloc[nodes, :].iloc[:, nodes].values.tolist()
    sub_demand = [full_demand_vector[i] for i in nodes]
    cluster_labels = [cluster_ids[i] for i in nodes]  # real cluster numbers for printing

    def create_data_model():
        return {
            'distance_matrix': sub_matrix,
            'demands': sub_demand,
            'vehicle_capacities': [16000] * 4,
            'num_vehicles': 4,
            'depot': 0
        }

    from ortools.constraint_solver import pywrapcp, routing_enums_pb2

    def print_solution(data, manager, routing, solution):
        total_distance = 0
        for vehicle_id in range(data['num_vehicles']):
            index = routing.Start(vehicle_id)
            route_distance = 0
            capacity_used = 0
            plan_output = f'\n🚚 Route for vehicle {vehicle_id} (Warehouse: Cluster {cluster_labels[0]}):\n'

            while not routing.IsEnd(index):
                node_index = manager.IndexToNode(index)
                demand = data['demands'][node_index]
                capacity_used += demand
                plan_output += f' → Cluster {cluster_labels[node_index]} (Demand: {demand}, Used: {capacity_used})\n'
                previous_index = index
                index = solution.Value(routing.NextVar(index))
                route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)

            plan_output += f' → Return to Cluster {cluster_labels[0]} (Depot)\n'
            plan_output += f'Distance: {route_distance} km\n'
            print(plan_output)
            total_distance += route_distance

        return total_distance

    def solve_cvrp():
        data = create_data_model()
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'],
                                               data['depot'])
        routing = pywrapcp.RoutingModel(manager)

        def distance_callback(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return int(data['distance_matrix'][from_node][to_node])
        transit_cb = routing.RegisterTransitCallback(distance_callback)
        routing.SetArcCostEvaluatorOfAllVehicles(transit_cb)

        # Add distance dimension
        routing.AddDimension(
            transit_cb,
            0,        # no slack
            6000,     # vehicle max travel distance (km)
            True,     # start cumul at zero
            'Distance'
        )
        distance_dimension = routing.GetDimensionOrDie('Distance')
        distance_dimension.SetGlobalSpanCostCoefficient(100)


        def demand_callback(from_index):
            return data['demands'][manager.IndexToNode(from_index)]
        demand_cb = routing.RegisterUnaryTransitCallback(demand_callback)

        routing.AddDimensionWithVehicleCapacity(
            demand_cb, 0, data['vehicle_capacities'], True, 'Capacity'
        )

        search_params = pywrapcp.DefaultRoutingSearchParameters()
        search_params.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        solution = routing.SolveWithParameters(search_params)
        return print_solution(data, manager, routing, solution) if solution else None

    total_distance = solve_cvrp()
    print(f"\n✅ Finished CVRP for warehouse Cluster {cluster_labels[0]}. Distance: {total_distance:.2f} km")
    return total_distance

In [18]:
total_distance_with_2_warehouses = 0
for wh_idx, assigned_nodes in assignments_by_wh.items():
    print(f"\n🚚 Running CVRP for warehouse index {wh_idx} (Cluster {clusters_to_keep[wh_idx]})")
    distance = run_cvrp_for_warehouse(
        wh_index=wh_idx,
        assigned_node_indices=assigned_nodes,
        full_distance_df=filtered_distance,
        full_demand_vector=filtered_demand,
        cluster_ids=clusters_to_keep
    )
    total_distance_with_2_warehouses += distance

print(f"\n📉 Total distance with 2 warehouses: {total_distance_with_2_warehouses:.2f} km")


🚚 Running CVRP for warehouse index 0 (Cluster 0)

🚚 Route for vehicle 0 (Warehouse: Cluster 0):
 → Cluster 0 (Demand: 0, Used: 0)
 → Return to Cluster 0 (Depot)
Distance: 0 km


🚚 Route for vehicle 1 (Warehouse: Cluster 0):
 → Cluster 0 (Demand: 0, Used: 0)
 → Cluster 36 (Demand: 162, Used: 162)
 → Cluster 23 (Demand: 46, Used: 208)
 → Cluster 52 (Demand: 91, Used: 299)
 → Cluster 26 (Demand: 161, Used: 460)
 → Return to Cluster 0 (Depot)
Distance: 5711 km


🚚 Route for vehicle 2 (Warehouse: Cluster 0):
 → Cluster 0 (Demand: 0, Used: 0)
 → Cluster 21 (Demand: 24, Used: 24)
 → Cluster 9 (Demand: 237, Used: 261)
 → Cluster 8 (Demand: 479, Used: 740)
 → Cluster 42 (Demand: 14, Used: 754)
 → Cluster 29 (Demand: 60, Used: 814)
 → Cluster 16 (Demand: 127, Used: 941)
 → Cluster 31 (Demand: 159, Used: 1100)
 → Cluster 28 (Demand: 23, Used: 1123)
 → Cluster 17 (Demand: 5, Used: 1128)
 → Cluster 12 (Demand: 101, Used: 1229)
 → Cluster 4 (Demand: 2919, Used: 4148)
 → Cluster 46 (Demand: 183, Use

# Evaluation

In [21]:
print(f"📦 Baseline distance: {baseline_total_distance}")
print(f"🏪 2-warehouse distance: {total_distance_with_2_warehouses}")

📦 Baseline distance: 17916
🏪 2-warehouse distance: 19721


In [28]:
improvement = total_distance_with_2_warehouses - baseline_total_distance
improvement_pct = improvement / baseline_total_distance * 100

print(f"\n📉 Distance changed by: {improvement:.2f} km ({improvement_pct:.2f}%)")


📉 Distance changed by: 1805.00 km (10.07%)
